# Import all the required libraries

In [166]:
import pandas as pd
import numpy as np
import yfinance as yf
import warnings
import numpy as np
import pandas as pd
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
idx= pd.IndexSlice
sns.set_style('whitegrid')
warnings.filterwarnings('ignore')
%matplotlib inline

# Get all the price data and extract them into variables

In [167]:
start_date = '2013-01-01'
end_date = '2020-07-12'

keys = ['TSLA' , 'GOOG' , 'META' , 'AAPL' , 'MSFT' , 'AMZN' , 'NVDA' , 'V' , 'ORCL' , 'CSCO']

open = pd.DataFrame(columns=keys)
high =pd.DataFrame(columns=keys)
low =pd.DataFrame(columns=keys)
close = pd.DataFrame(columns=keys)
volume = pd.DataFrame(columns=keys)
vwap = pd.DataFrame(columns=keys)
returns = pd.DataFrame(columns=keys)

for s in keys:
    temp = yf.download(s,
                               start_date,
                               end_date)
    #   temp.to_hdf('data.h5' , key=s)
    open[s] = temp['Open'].replace(np.nan ,0)
    high[s] = temp['High'].replace(np.nan ,0)
    low[s] = temp['Low'].replace(np.nan ,0)
    close[s] = temp['Close'].replace(np.nan ,0)
    volume[s] = temp['Volume'].replace(np.nan ,0)
    vwap[s] = ((temp['High'] + temp['Low'] + temp['Close'])/3).replace(np.nan ,0)
    returns[s] = temp.Close.pct_change().replace(np.nan ,0)

open.fillna(0 , inplace=True)
print('open -' , open.shape)
high.fillna(0 , inplace=True)
print('high -' , high.shape)
low.fillna(0 , inplace=True)
print('low -' , low.shape)
close.fillna(0 , inplace=True)
print('close -' , close.shape)
volume.fillna(0 , inplace=True)
print('volume -' , volume.shape)
vwap.fillna(0 , inplace=True)
print('vwap -' , vwap.shape)
returns.fillna(0 , inplace=True)
print('returns -' , returns.shape)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
open - (1895, 10)
high - (1895, 10)
low - (1895, 10)
close - (1895, 10)
volume - (1895, 10)
vwap - (1895, 10)
returns - (1895, 10)


In [168]:
open.tail()

,TSLA,GOOG,META,AAPL,MSFT,AMZN,NVDA,V,ORCL,CSCO
Date,,,,,,,,,,
2020-07-06,85.112663,74.002998,233.759995,92.500000,208.830002,146.748505,97.417503,198.729996,56.200001,46.070000
2020-07-07,93.667336,74.500000,239.410004,93.852501,210.449997,152.927505,99.462502,196.809998,56.349998,46.119999
2020-07-08,93.666664,74.716003,238.110001,94.180000,210.070007,151.130493,99.937500,195.080002,56.630001,45.590000
2020-07-09,93.132668,75.322502,245.000000,96.262497,216.330002,155.799500,103.802498,195.000000,56.990002,46.599998
2020-07-10,93.066666,75.307503,243.690002,95.334999,213.619995,159.587997,105.887497,191.649994,57.470001,46.730000


In [169]:
close.tail()

,TSLA,GOOG,META,AAPL,MSFT,AMZN,NVDA,V,ORCL,CSCO
Date,,,,,,,,,,
2020-07-06,91.438667,74.785004,240.279999,93.462502,210.699997,152.852005,98.392502,197.759995,56.599998,46.419998
2020-07-07,92.657333,74.259003,240.860001,93.172501,208.250000,150.005997,98.717499,194.199997,56.310001,45.630001
2020-07-08,91.058670,74.800003,243.580002,95.342499,212.830002,154.055496,102.160004,195.070007,56.660000,45.810001
2020-07-09,92.952003,75.549500,244.500000,95.752502,214.320007,159.131500,105.089996,192.210007,57.529999,46.700001
2020-07-10,102.976669,77.086998,245.070007,95.919998,213.669998,160.000000,104.792503,192.550003,57.389999,46.660000


In [170]:
returns.tail()

,TSLA,GOOG,META,AAPL,MSFT,AMZN,NVDA,V,ORCL,CSCO
Date,,,,,,,,,,
2020-07-06,0.134794,0.021165,0.029389,0.026750,0.021526,0.057690,0.023616,0.010681,0.011798,0.017313
2020-07-07,0.013328,-0.007034,0.002414,-0.003103,-0.011628,-0.018619,0.003303,-0.018002,-0.005124,-0.017018
2020-07-08,-0.017254,0.007285,0.011293,0.023290,0.021993,0.026996,0.034872,0.004480,0.006216,0.003945
2020-07-09,0.020792,0.010020,0.003777,0.004300,0.007001,0.032949,0.028680,-0.014661,0.015355,0.019428
2020-07-10,0.107848,0.020351,0.002331,0.001749,-0.003033,0.005458,-0.002831,0.001769,-0.002434,-0.000857


In [171]:
def get_data(comp):
    return pd.read_csv('data.h5' , comp)

# Defining operations to make alphas

In [172]:
def rank(df):
    """Return the cross-sectional percentile rank

     Args:
         :param df: tickers in columns, sorted dates in rows.

     Returns:
         pd.DataFrame: the ranked values
     """
    return df.rank(axis=1, pct=True)

def scale(df):
    """
    Scaling time serie.
    :param df: a pandas DataFrame.
    :param k: scaling factor.
    :return: a pandas DataFrame rescaled df such that sum(abs(df)) = k
    """
    return df.div(df.abs().sum(axis=1), axis=0)

def normalize(df):
    return df.apply(lambda x: (x - x.min()) / (x.max() - x.min()) * 2 - 1, axis=1)


def log(df):
    return np.log1p(df)

def sign(df):
    return np.sign(df)

def power(df, exp):
    return df.pow(exp)

def ts_lag(df: pd.DataFrame, t: int = 1) -> pd.DataFrame:
    """Return the lagged values t periods ago.

    Args:
        :param df: tickers in columns, sorted dates in rows.
        :param t: lag

    Returns:
        pd.DataFrame: the lagged values
    """
    return df.shift(t)

def ts_delta(df, period=1):
    """
    Wrapper function to estimate difference.
    :param df: a pandas DataFrame.
    :param period: the difference grade.
    :return: a pandas DataFrame with today’s value minus the value 'period' days ago.
    """
    return df.diff(period)

def ts_sum(df: pd.DataFrame, window: int = 10) -> pd.DataFrame:
    """Computes the rolling ts_sum for the given window size.

    Args:
        df (pd.DataFrame): tickers in columns, dates in rows.
        window      (int): size of rolling window.

    Returns:
        pd.DataFrame: the ts_sum over the last 'window' days.
    """
    return df.rolling(window).sum()

def ts_mean(df, window=10):
    """Computes the rolling mean for the given window size.

    Args:
        df (pd.DataFrame): tickers in columns, dates in rows.
        window      (int): size of rolling window.

    Returns:
        pd.DataFrame: the mean over the last 'window' days.
    """
    return df.rolling(window).mean()

def ts_std(df, window=10):
    """
    Wrapper function to estimate rolling standard deviation.
    :param df: a pandas DataFrame.
    :param window: the rolling window.
    :return: a pandas DataFrame with the time-series min over the past 'window' days.
    """
    return (df
            .rolling(window)
            .std())

def ts_rank(df, window=10):
    """
    Wrapper function to estimate rolling rank.
    :param df: a pandas DataFrame.
    :param window: the rolling window.
    :return: a pandas DataFrame with the time-series rank over the past window days.
    """
    return (df
            .rolling(window)
            .apply(lambda x: x.rank().iloc[-1]))

def ts_product(df, window=10):
    """
    Wrapper function to estimate rolling ts_product.
    :param df: a pandas DataFrame.
    :param window: the rolling window.
    :return: a pandas DataFrame with the time-series ts_product over the past 'window' days.
    """
    return (df
            .rolling(window)
            .apply(np.prod))


def ts_min(df, window=10):
    """
    Wrapper function to estimate rolling min.
    :param df: a pandas DataFrame.
    :param window: the rolling window.
    :return: a pandas DataFrame with the time-series min over the past 'window' days.
    """
    return df.rolling(window).min()


def ts_max(df, window=10):
    """
    Wrapper function to estimate rolling min.
    :param df: a pandas DataFrame.
    :param window: the rolling window.
    :return: a pandas DataFrame with the time-series max over the past 'window' days.
    """
    return df.rolling(window).max()


def ts_argmax(df, window=10):
    """
    Wrapper function to estimate which day ts_max(df, window) occurred on
    :param df: a pandas DataFrame.
    :param window: the rolling window.
    :return: well.. that :)
    """
    return df.rolling(window).apply(np.argmax).add(1)

def ts_argmin(df, window=10):
    """
    Wrapper function to estimate which day ts_min(df, window) occurred on
    :param df: a pandas DataFrame.
    :param window: the rolling window.
    :return: well.. that :)
    """
    return (df.rolling(window)
            .apply(np.argmin)
            .add(1))

def ts_corr(x, y, window=10):
    """
    Wrapper function to estimate rolling correlations.
    :param x, y: pandas DataFrames.
    :param window: the rolling window.
    :return: a pandas DataFrame with the time-series min over the past 'window' days.
    """
    return x.rolling(window).corr(y)

def ts_cov(x, y, window=10):
    """
    Wrapper function to estimate rolling covariance.
    :param df: a pandas DataFrame.
    :param window: the rolling window.
    :return: a pandas DataFrame with the time-series min over the past 'window' days.
    """
    return x.rolling(window).cov(y)

# Creating alphas

In [173]:
def alpha1(o ,h ,l , c , vwap , v , r):
    """(-1 * ts_corr(rank(ts_delta(log(volume), 2)), rank(((close - open) / open)), 6))"""
    s1 = rank(ts_delta(log(v), 2))
    s2 = rank((c / o) - 1)
    alpha = ts_corr(s1, s2, 6)
    return alpha

alpha1(open , high , low , close , vwap , volume , returns).dropna()

,TSLA,GOOG,META,AAPL,MSFT,AMZN,NVDA,V,ORCL,CSCO
Date,,,,,,,,,,
2013-01-10,-0.445789,0.424596,0.862764,0.199415,5.801423e-01,0.373036,0.093554,-0.102003,-0.130455,0.427283
2013-01-11,-0.597790,0.594909,0.819311,0.130774,7.572402e-01,0.740120,0.447121,0.070392,-0.289366,0.245638
2013-01-14,-0.181949,-0.286377,0.572689,0.495284,5.344984e-01,0.821784,-0.353094,0.350456,-0.438286,0.632860
2013-01-15,-0.386577,0.021190,0.339879,-0.330911,7.378785e-02,0.738260,-0.161515,0.410171,-0.584127,0.148250
2013-01-16,-0.094095,0.251433,0.335790,0.013834,-2.498244e-02,0.823098,0.828647,0.209585,-0.461593,0.078286
...,...,...,...,...,...,...,...,...,...,...
2020-07-06,0.431321,0.057735,-0.380509,0.196195,-3.770096e-01,0.135597,0.679250,-0.711438,-0.409718,0.692345
2020-07-07,-0.144338,0.426798,-0.063133,0.489051,-6.558113e-01,0.497519,0.774597,-0.839599,-0.348868,0.020870
2020-07-08,0.595946,0.454946,-0.169195,0.735730,-6.451792e-01,0.406138,0.864586,-0.879778,-0.333586,0.812421


In [174]:
def alpha2(o ,h ,l , c , vwap ,v , r):
    """(-1 * ts_corr(rank(open), rank(volume), 10))"""
    o = pd.DataFrame((o))
    v =  pd.DataFrame((v))
    return ts_corr(o,v, 10)

alpha2(open , high , low , close , vwap , volume , returns).dropna()

,TSLA,GOOG,META,AAPL,MSFT,AMZN,NVDA,V,ORCL,CSCO
Date,,,,,,,,,,
2013-01-14,0.019785,-0.268928,0.638983,-0.406678,-0.067650,-0.003166,0.443316,0.056977,-0.545773,0.362078
2013-01-15,-0.431919,-0.713825,0.497180,-0.574391,-0.157477,0.024205,0.387748,-0.007793,-0.616053,0.161548
2013-01-16,-0.545701,-0.650770,0.444211,-0.728387,-0.258261,-0.020332,0.486293,0.139694,-0.285191,0.281855
2013-01-17,-0.460016,-0.565787,0.325262,-0.598675,-0.266075,-0.167278,0.534853,0.206481,-0.359969,0.416000
2013-01-18,0.237535,-0.658153,0.284423,-0.602450,-0.322649,-0.661935,0.545567,0.172708,-0.122014,0.479896
...,...,...,...,...,...,...,...,...,...,...
2020-07-06,0.810036,-0.098289,-0.544459,0.077879,-0.025692,0.776600,-0.035970,-0.139277,-0.099058,-0.351785
2020-07-07,0.869179,-0.175709,-0.529020,-0.199548,-0.042741,0.535070,-0.104406,-0.279514,-0.422419,-0.447767
2020-07-08,0.821685,-0.240972,-0.507080,-0.296174,-0.049676,0.423242,-0.133744,-0.354813,-0.636422,-0.414888


In [175]:
def alpha3(o ,h ,l , c , vwap ,v , r):
    """(rank((open - ts_mean(vwap, 10))) * (-1 * abs(rank((close - vwap)))))"""
    return -normalize(rank(o.sub(ts_mean(vwap, 10))).mul(rank(c.sub(vwap)).mul(-1).abs()))


alpha3(open , high , low , close , vwap , volume , returns).dropna()

,TSLA,GOOG,META,AAPL,MSFT,AMZN,NVDA,V,ORCL,CSCO
Date,,,,,,,,,,
2013-01-14,0.714286,0.818182,0.818182,1.000000,0.662338,-0.168831,6.103896e-01,-0.090909,-1.000000,-0.558442
2013-01-15,0.342857,0.714286,0.771429,1.000000,-0.085714,-0.542857,8.000000e-01,-0.742857,0.371429,-1.000000
2013-01-16,-0.034483,1.000000,0.517241,0.724138,0.137931,-0.137931,3.448276e-01,-0.793103,0.379310,-1.000000
2013-01-17,-0.117647,0.970588,0.661765,1.000000,-0.529412,-0.588235,-2.220446e-16,0.470588,0.264706,-1.000000
2013-01-18,0.088235,1.000000,0.941176,0.941176,-0.941176,0.411765,5.294118e-01,0.323529,-0.058824,-1.000000
...,...,...,...,...,...,...,...,...,...,...
2020-07-06,-1.000000,0.477273,0.363636,0.772727,-0.227273,-0.590909,5.000000e-01,0.090909,1.000000,1.000000
2020-07-07,-1.000000,0.782609,0.869565,0.565217,1.000000,0.391304,5.217391e-01,0.847826,0.782609,1.000000
2020-07-08,0.558442,0.974026,-0.558442,0.428571,-1.000000,-0.792208,-1.298701e-02,0.610390,0.870130,1.000000


In [176]:
def alpha4(o ,h ,l , c , vwap ,v , r):
    """(-ts_corr(open, volume, 10))"""
    return  ts_corr(o, v, 10)

alpha4(open , high , low , close , vwap , volume , returns).dropna()

,TSLA,GOOG,META,AAPL,MSFT,AMZN,NVDA,V,ORCL,CSCO
Date,,,,,,,,,,
2013-01-14,0.019785,-0.268928,0.638983,-0.406678,-0.067650,-0.003166,0.443316,0.056977,-0.545773,0.362078
2013-01-15,-0.431919,-0.713825,0.497180,-0.574391,-0.157477,0.024205,0.387748,-0.007793,-0.616053,0.161548
2013-01-16,-0.545701,-0.650770,0.444211,-0.728387,-0.258261,-0.020332,0.486293,0.139694,-0.285191,0.281855
2013-01-17,-0.460016,-0.565787,0.325262,-0.598675,-0.266075,-0.167278,0.534853,0.206481,-0.359969,0.416000
2013-01-18,0.237535,-0.658153,0.284423,-0.602450,-0.322649,-0.661935,0.545567,0.172708,-0.122014,0.479896
...,...,...,...,...,...,...,...,...,...,...
2020-07-06,0.810036,-0.098289,-0.544459,0.077879,-0.025692,0.776600,-0.035970,-0.139277,-0.099058,-0.351785
2020-07-07,0.869179,-0.175709,-0.529020,-0.199548,-0.042741,0.535070,-0.104406,-0.279514,-0.422419,-0.447767
2020-07-08,0.821685,-0.240972,-0.507080,-0.296174,-0.049676,0.423242,-0.133744,-0.354813,-0.636422,-0.414888


In [177]:
def alpha5(o ,h ,l , c , vwap ,v , r):
    return normalize(ts_delta(c,2))

alpha5(open , high , low , close , vwap , volume , returns).dropna()

,TSLA,GOOG,META,AAPL,MSFT,AMZN,NVDA,V,ORCL,CSCO
Date,,,,,,,,,,
2013-01-03,-1.000000,-0.365634,1.000000,-0.459371,-0.119264,-0.411927,-0.894496,0.688073,0.706426,0.357801
2013-01-04,-0.004877,0.520277,1.000000,-0.886322,-1.000000,0.185366,0.204268,0.496951,-0.024387,0.243902
2013-01-07,-0.459711,-0.194812,1.000000,-1.000000,-0.921739,-0.000869,-0.426087,0.084783,-0.330436,-0.573913
2013-01-08,-0.660894,-0.824032,0.170149,-0.690406,-1.000000,0.317016,-0.940297,1.000000,-0.952242,-0.952238
2013-01-09,-0.722278,-0.537403,1.000000,-1.000000,-0.642063,-0.805562,-0.854399,0.723961,-0.613753,-0.642065
...,...,...,...,...,...,...,...,...,...,...
2020-07-06,1.000000,-0.693003,-0.711621,-0.747515,-0.313729,0.041210,-0.667512,-0.559522,-0.908741,-1.000000
2020-07-07,1.000000,-0.631882,0.315129,-0.466420,-0.489298,0.027453,-0.400000,-1.000000,-0.728413,-0.783026
2020-07-08,-0.284555,-0.162217,0.855207,0.415406,0.492838,0.205880,1.000000,-1.000000,-0.148279,-0.355790


In [178]:
def alpha6(o ,h ,l , c , vwap ,v , r):
    """-rank(((ts_sum(open, 5) * ts_sum(returns, 5)) -
        ts_lag((ts_sum(open, 5) * ts_sum(returns, 5)),10)))
    """
    return normalize((rank(((ts_sum(o, 5) * ts_sum(r, 5)) -
                       ts_lag((ts_sum(o, 5) * ts_sum(r, 5)), 10)))))

alpha6(open , high , low , close , vwap , volume , returns).dropna()

,TSLA,GOOG,META,AAPL,MSFT,AMZN,NVDA,V,ORCL,CSCO
Date,,,,,,,,,,
2013-01-22,0.555556,-0.555556,-1.000000,1.000000,0.777778,-0.111111,0.333333,-0.777778,-0.333333,0.111111
2013-01-23,0.555556,0.111111,-0.777778,1.000000,0.777778,-0.111111,0.333333,-1.000000,-0.555556,-0.333333
2013-01-24,0.333333,0.777778,-0.777778,-0.555556,1.000000,-0.333333,0.111111,-1.000000,0.555556,-0.111111
2013-01-25,0.333333,0.777778,-1.000000,-0.777778,1.000000,0.555556,-0.333333,-0.555556,-0.111111,0.111111
2013-01-28,0.333333,1.000000,-0.333333,-0.777778,0.777778,-0.555556,0.555556,-1.000000,0.111111,-0.111111
...,...,...,...,...,...,...,...,...,...,...
2020-07-06,1.000000,-0.111111,0.777778,-0.555556,0.111111,0.555556,-0.333333,0.333333,-1.000000,-0.777778
2020-07-07,1.000000,0.111111,0.777778,-0.777778,-1.000000,0.555556,0.333333,-0.333333,-0.555556,-0.111111
2020-07-08,1.000000,0.111111,0.777778,-0.555556,-0.111111,0.555556,0.333333,-1.000000,-0.333333,-0.777778


In [179]:
def alpha7(o ,h ,l , c , vwap ,v , r):
    """(sign(ts_delta(volume, 1)) *
            (-1 * ts_delta(close, 1)))
        """
    return normalize(sign(ts_delta(v, 1)).mul(ts_delta(c, 1)))

alpha7(open , high , low , close , vwap , volume , returns).dropna()

,TSLA,GOOG,META,AAPL,MSFT,AMZN,NVDA,V,ORCL,CSCO
Date,,,,,,,,,,
2013-01-02,-0.292205,0.006396,1.000000,-1.000000,0.525768,-0.717334,-0.276396,0.566127,0.989910,0.303785
2013-01-03,-0.553784,-0.780899,0.315855,0.395672,0.954406,-1.000000,-0.744584,-0.870008,1.000000,-0.231465
2013-01-04,-0.262462,0.171071,1.000000,-1.000000,-0.961701,-0.338534,-0.157403,-0.709950,-0.687061,-0.333954
2013-01-07,-0.395747,-0.233622,1.000000,-0.168695,-0.297876,0.586170,-0.606385,-1.000000,-0.021277,-0.000004
2013-01-08,-0.621672,-0.590600,0.970445,-0.646719,-1.000000,-0.038421,-0.172411,1.000000,-0.408870,-0.369461
...,...,...,...,...,...,...,...,...,...,...
2020-07-06,1.000000,-0.400723,-1.000000,0.049019,0.275299,0.715108,-0.481981,0.010081,-0.300278,-0.314950
2020-07-07,0.491932,0.275677,0.292537,0.201997,-0.653450,1.000000,0.212923,-1.000000,0.201995,-0.135186
2020-07-08,0.540333,0.006918,0.819881,0.682766,-1.000000,-0.867746,1.000000,0.358681,0.054534,0.096914


In [180]:
def alpha8(o ,h ,l , c , vwap ,v , r):
    """-rank(ts_cov(rank(close), rank(volume), 5))"""
    return normalize(rank(ts_cov(rank(c), rank(v), 5)))

alpha8(open , high , low , close , vwap , volume , returns).dropna()

,TSLA,GOOG,META,AAPL,MSFT,AMZN,NVDA,V,ORCL,CSCO
Date,,,,,,,,,,
2013-01-07,-0.777778,-0.111111,1.000000,-0.111111,0.777778,0.555556,-0.111111,-0.111111,-0.111111,-1.000000
2013-01-08,0.111111,-0.444444,1.000000,-0.444444,0.777778,0.444444,0.444444,-0.444444,-0.444444,-1.000000
2013-01-09,0.777778,-0.666667,-1.000000,0.111111,0.111111,1.000000,0.111111,0.111111,0.111111,-0.666667
2013-01-10,0.454545,-1.000000,-1.000000,-1.000000,1.000000,-1.000000,-1.000000,-1.000000,1.000000,-1.000000
2013-01-11,-0.777778,-1.000000,1.000000,-0.222222,-0.222222,0.666667,0.333333,-0.222222,0.666667,-0.222222
...,...,...,...,...,...,...,...,...,...,...
2020-07-06,-0.333333,-1.000000,0.666667,-0.555556,-0.111111,-0.777778,0.666667,0.333333,0.111111,1.000000
2020-07-07,-0.444444,0.444444,0.444444,-1.000000,-0.444444,-0.777778,0.444444,0.444444,-0.111111,1.000000
2020-07-08,-0.647059,0.176471,0.647059,-1.000000,-0.411765,-1.000000,0.647059,0.176471,-0.176471,1.000000


In [181]:
def alpha9(o ,h ,l , c , vwap ,v , r):
    """
    (rank(ts_delta(returns, 3))) * ts_corr(open, volume, 10))
    """

    alpha = -rank(ts_delta(r, 3)).mul(ts_corr(o, v, 10)
                                      .replace([-np.inf,
                                                np.inf],
                                               np.nan))
    return normalize(alpha)

alpha9(open , high , low , close , vwap , volume , returns).dropna()

,TSLA,GOOG,META,AAPL,MSFT,AMZN,NVDA,V,ORCL,CSCO
Date,,,,,,,,,,
2013-01-14,0.060323,0.246530,-0.077468,0.434342,0.191576,0.105425,-1.000000,0.035706,1.000000,-0.898392
2013-01-15,1.000000,0.987654,-0.640961,-0.080173,0.011309,-0.529297,-1.000000,-0.450328,0.158108,-0.746770
2013-01-16,0.575350,0.162215,-0.383200,1.000000,-0.165020,-0.296415,-1.000000,-0.428708,0.002574,-0.656825
2013-01-17,0.222953,0.811262,-0.892390,1.000000,0.169599,-0.106766,-1.000000,-0.406143,0.056472,-0.271144
2013-01-18,-0.272323,0.177111,-0.686374,1.000000,-0.092948,0.582894,-1.000000,-0.364424,-0.000942,-0.711587
...,...,...,...,...,...,...,...,...,...,...
2020-07-06,-1.000000,0.721344,0.906507,0.464553,0.596649,-0.782400,0.593629,0.695702,0.683595,1.000000
2020-07-07,-1.000000,0.134418,0.284842,0.493910,0.057726,-0.166537,0.280881,0.381043,0.822861,1.000000
2020-07-08,-0.466979,-0.142431,1.000000,0.327464,-0.188211,-1.000000,0.037255,0.002585,0.371354,0.254033


In [182]:
def alpha10(o ,h ,l , c , vwap ,v , r):
    """-rank(open - ts_lag(high, 1)) *
        rank(open - ts_lag(close, 1)) *
        rank(open -ts_lag(low, 1))"""
    return -normalize(rank(o - ts_lag(h, 1)).mul(rank(o - ts_lag(c, 1))).mul(rank(o - ts_lag(l, 1))).mul(-1))

alpha10(open , high , low , close , vwap , volume , returns).dropna()

,TSLA,GOOG,META,AAPL,MSFT,AMZN,NVDA,V,ORCL,CSCO
Date,,,,,,,,,,
2013-01-02,-0.984427,-0.895439,-0.067853,1.000000,-0.441602,-0.922136,-1.000000,0.259177,0.136819,-0.724138
2013-01-03,-0.957704,-0.691843,-1.000000,-0.953676,-0.562941,-0.832830,-0.820745,1.000000,-0.905337,-0.035247
2013-01-04,-0.759162,1.000000,-0.312391,-1.000000,-0.549738,-0.842932,-0.486911,-0.839442,0.246073,-0.607330
2013-01-07,-0.214018,-0.476846,-0.501877,-1.000000,-0.842303,1.000000,-0.281602,0.416771,-0.957447,-0.942428
2013-01-08,-0.521691,-0.348165,-0.401557,1.000000,-0.468298,-0.735261,-0.735261,-0.877642,-1.000000,-0.966630
...,...,...,...,...,...,...,...,...,...,...
2020-07-06,1.000000,-0.765296,-0.993982,-0.717151,0.221665,-0.809428,-0.654965,0.221665,-1.000000,-0.957874
2020-07-07,1.000000,-0.935743,-0.935743,-0.445783,-0.698795,-0.108434,-0.032129,-1.000000,-0.952811,-0.935743
2020-07-08,0.586364,-0.568182,-1.000000,0.222727,-0.186364,0.159091,1.000000,-0.595455,-0.595455,-0.859091


# Analyzing the result and getting a benchmark for the model

In [183]:
import pandas as pd

def apply_functions_and_sum(o ,h ,l , c , vwap ,v , r, function_list , comp):
    result_sum = pd.DataFrame()
    for i , func in enumerate(function_list):
        func_result = func(o ,h ,l , c , vwap ,v , r)
        result_sum['A_'+str(i)] = func_result[comp]
    return result_sum

function_list = [alpha1 , alpha2 ,alpha3 , alpha4 , alpha5 , alpha6 , alpha7 , alpha8 , alpha9 , alpha10]

for col in list(open.columns):
    name = col
    
#     temp.to_hdf('data.h5' , key=(f'res_'+str(col)))
    if col == 'TSLA':
        temp = apply_functions_and_sum(open , high , low , close , vwap , volume , returns, function_list , col)
        
        temp.dropna(inplace=True)
        tesla = temp.copy()
        temp = temp.sum(axis=1)
        print(temp)
#     if col == 'TSLA':
#         tesla = temp.copy()
#     print(f'{col} = {temp.mul((open[name] - close[name])/ open[name]).abs().sum(axis=0)*100} % ')
    
#     if temp is -ve ( short signal ) and stock decreases => profit ( +ve )
#                                                  and stock increases => loss ( -ve )           
#     if temp is +ve ( short signal ) and stock decreases => loss ( -ve )
#                                                  and stock increases => profit (+ve ) 

Date
2013-01-22   -2.179631
2013-01-23   -2.556536
2013-01-24   -1.050159
2013-01-25    0.601412
2013-01-28   -0.681212
                ...   
2020-07-06    3.718060
2020-07-07    2.641508
2020-07-08    3.525862
2020-07-09   -1.223796
2020-07-10    1.556521
Length: 1881, dtype: float64


In [205]:
(tesla.sum(axis=1)*((open['TSLA'] - close['TSLA'])/open['TSLA'])[14:]).sum(axis=0)

-6.914955301504666

In [185]:
for col in returns.columns:
    print(((open[col] - close[col])/open[col]).abs().sum()*100)

3636.584537504046
1651.1101157771066
2248.8507038230614
1777.9161214878186
1650.1316169921279
2023.6615958570248
2644.2546165121976
1552.7360106895346
1540.6323370771652
1574.9457672822107


In [186]:
for col in returns.columns:
    print(f' {col} = {returns[col].abs().sum()*100} % , high = {returns[col].abs().max()} , min = {returns[col].abs().min()}')

 TSLA = 4341.738260439361 % , high = 0.24395075824616308 , min = 0.0
 GOOG = 2012.623409856938 % , high = 0.16052424867207615 , min = 0.0
 META = 2680.620106055167 % , high = 0.2961146855410959 , min = 0.0
 AAPL = 2247.2103832619764 % , high = 0.12864695596573805 , min = 0.0
 MSFT = 2050.1842410724576 % , high = 0.1473903128642552 , min = 0.0
 AMZN = 2456.3166617970514 % , high = 0.1413112565602248 , min = 0.0
 NVDA = 3187.5292549452697 % , high = 0.2980670736052544 , min = 0.0
 V = 1945.442112092097 % , high = 0.13842634835519507 , min = 0.0
 ORCL = 1858.032172689129 % , high = 0.20427138753673213 , min = 0.0
 CSCO = 1970.4754286178145 % , high = 0.1337348953216071 , min = 0.0


In [ ]:
tesla = pd.read_hdf('data.h5' , 'res_TSLA')

# Analyze alpha result wrt company and creating target variable

In [187]:
tesla.columns

Index(['A_0', 'A_1', 'A_2', 'A_3', 'A_4', 'A_5', 'A_6', 'A_7', 'A_8', 'A_9'], dtype='object')

In [188]:
y_pred_col = ['y_'+i for i in list(tesla.columns)]

In [189]:
pred_tesla = tesla.copy()
for col in list(tesla.columns):
    pred_tesla[col] = pred_tesla[col].mul((open['TSLA'] - close['TSLA'])/open['TSLA'])
pred_tesla.dropna(inplace=True)

In [192]:
tesla.tail()

,A_0,A_1,A_2,A_3,A_4,A_5,A_6,A_7,A_8,A_9
Date,,,,,,,,,,
2020-07-06,0.431321,0.810036,-1.000000,0.810036,1.000000,1.0,1.000000,-0.333333,-1.000000,1.000000
2020-07-07,-0.144338,0.869179,-1.000000,0.869179,1.000000,1.0,0.491932,-0.444444,-1.000000,1.000000
2020-07-08,0.595946,0.821685,0.558442,0.821685,-0.284555,1.0,0.540333,-0.647059,-0.466979,0.586364
2020-07-09,0.390352,0.635039,-0.721519,0.635039,-0.588923,1.0,-0.756384,-0.555556,-0.562967,-0.698876
2020-07-10,0.400870,0.633381,-1.000000,0.633381,1.000000,1.0,1.000000,-0.777778,-1.000000,-0.333333


In [193]:
pred_tesla.tail()

,A_0,A_1,A_2,A_3,A_4,A_5,A_6,A_7,A_8,A_9
Date,,,,,,,,,,
2020-07-06,-0.032058,-0.060206,0.074325,-0.060206,-0.074325,-0.074325,-0.074325,0.024775,0.074325,-0.074325
2020-07-07,-0.001556,0.009372,-0.010783,0.009372,0.010783,0.010783,0.005304,-0.004792,-0.010783,0.010783
2020-07-08,0.016593,0.022878,0.015549,0.022878,-0.007923,0.027843,0.015045,-0.018016,-0.013002,0.016326
2020-07-09,0.000757,0.001232,-0.001400,0.001232,-0.001142,0.001940,-0.001467,-0.001078,-0.001092,-0.001356
2020-07-10,-0.042686,-0.067444,0.106483,-0.067444,-0.106483,-0.106483,-0.106483,0.082820,0.106483,0.035494


In [157]:
returns['TSLA']

Date
2012-12-31    0.000000
2013-01-02    0.043992
2013-01-03   -0.016685
2013-01-04   -0.010642
2013-01-07   -0.001744
                ...   
2020-07-06    0.134794
2020-07-07    0.013328
2020-07-08   -0.017254
2020-07-09    0.020792
2020-07-10    0.107848
Name: TSLA, Length: 1895, dtype: float64

In [208]:
temp = (pred_tesla.apply(lambda row: [1 if value == row.max() else 0 for value in row], axis=1))
pred_tesla = pd.DataFrame(temp.tolist(), columns=y_pred_col)
pred_tesla.set_index(tesla.index , inplace=True)
pred_tesla

,y_A_0,y_A_1,y_A_2,y_A_3,y_A_4,y_A_5,y_A_6,y_A_7,y_A_8,y_A_9
Date,,,,,,,,,,
2013-01-22,0,0,0,0,0,0,0,0,1,0
2013-01-23,0,0,0,0,0,0,0,0,1,0
2013-01-24,0,0,0,0,0,0,0,0,1,0
2013-01-25,0,0,0,0,1,0,0,0,0,0
2013-01-28,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
2020-07-06,0,0,1,0,0,0,0,0,1,0
2020-07-07,0,0,0,0,1,1,0,0,0,1
2020-07-08,0,0,0,0,0,1,0,0,0,0


In [154]:
result_data = [[(tesla.iloc[i, j]) if pred_tesla.iloc[i, j] == 1 else 0 for j in range(len(pred_tesla.columns))] for i in range(len(pred_tesla))]
result_df = pd.DataFrame(result_data , columns=pred_tesla.columns)
result_df.set_index(pred_tesla.index , inplace=True)
result_df

,y_A_0,y_A_1,y_A_2,y_A_3,y_A_4,y_A_5,y_A_6,y_A_7,y_A_8,y_A_9
Date,,,,,,,,,,
2013-01-22,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,-1.0,0.000000
2013-01-23,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,-1.0,0.000000
2013-01-24,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,-1.0,0.000000
2013-01-25,0.0,0.0,0.0,0.0,0.576541,0.0,0.0,0.0,0.0,0.000000
2013-01-28,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,-0.915966
...,...,...,...,...,...,...,...,...,...,...
2020-07-06,0.0,0.0,-1.0,0.0,0.000000,0.0,0.0,0.0,-1.0,0.000000
2020-07-07,0.0,0.0,0.0,0.0,1.000000,1.0,0.0,0.0,0.0,1.000000
2020-07-08,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.000000


In [206]:
((result_df.max(axis=1) + result_df.min(axis=1))*((open['TSLA'] - close['TSLA'])/open['TSLA'])[14:]).sum(axis=0)

32.24486391870985

In [ ]:
pred_tesla.set_index(tesla.index , inplace=True)

In [209]:
temp_tesla = pd.concat([tesla , pred_tesla ] , axis =1 )
temp_tesla

,A_0,A_1,A_2,A_3,A_4,A_5,A_6,A_7,A_8,A_9,y_A_0,y_A_1,y_A_2,y_A_3,y_A_4,y_A_5,y_A_6,y_A_7,y_A_8,y_A_9
Date,,,,,,,,,,,,,,,,,,,,
2013-01-22,-0.585260,0.531981,0.548387,0.531981,-0.339226,0.555556,-0.762320,-7.777778e-01,-1.000000,-0.882952,0,0,0,0,0,0,0,0,1,0
2013-01-23,-0.678095,0.506168,-0.062500,0.506168,-0.343458,0.555556,-0.429030,-6.470588e-01,-1.000000,-0.964286,0,0,0,0,0,0,0,0,1,0
2013-01-24,-0.678095,0.476078,-0.215190,0.476078,0.279617,0.333333,0.749645,-7.777778e-01,-1.000000,-0.693848,0,0,0,0,0,0,0,0,1,0
2013-01-25,-0.205853,0.168249,0.555556,0.168249,0.576541,0.333333,-0.088115,-1.111111e-01,0.007737,-0.803175,0,0,0,0,1,0,0,0,0,0
2013-01-28,-0.206203,0.191361,0.583333,0.191361,-0.204715,0.333333,0.045768,2.220446e-16,-0.699485,-0.915966,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-06,0.431321,0.810036,-1.000000,0.810036,1.000000,1.000000,1.000000,-3.333333e-01,-1.000000,1.000000,0,0,1,0,0,0,0,0,1,0
2020-07-07,-0.144338,0.869179,-1.000000,0.869179,1.000000,1.000000,0.491932,-4.444444e-01,-1.000000,1.000000,0,0,0,0,1,1,0,0,0,1
2020-07-08,0.595946,0.821685,0.558442,0.821685,-0.284555,1.000000,0.540333,-6.470588e-01,-0.466979,0.586364,0,0,0,0,0,1,0,0,0,0


In [210]:
for col in y_pred_col:
    temp_tesla[col] = temp_tesla[col].shift(-1)

In [211]:
temp_tesla

,A_0,A_1,A_2,A_3,A_4,A_5,A_6,A_7,A_8,A_9,y_A_0,y_A_1,y_A_2,y_A_3,y_A_4,y_A_5,y_A_6,y_A_7,y_A_8,y_A_9
Date,,,,,,,,,,,,,,,,,,,,
2013-01-22,-0.585260,0.531981,0.548387,0.531981,-0.339226,0.555556,-0.762320,-7.777778e-01,-1.000000,-0.882952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2013-01-23,-0.678095,0.506168,-0.062500,0.506168,-0.343458,0.555556,-0.429030,-6.470588e-01,-1.000000,-0.964286,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2013-01-24,-0.678095,0.476078,-0.215190,0.476078,0.279617,0.333333,0.749645,-7.777778e-01,-1.000000,-0.693848,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2013-01-25,-0.205853,0.168249,0.555556,0.168249,0.576541,0.333333,-0.088115,-1.111111e-01,0.007737,-0.803175,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2013-01-28,-0.206203,0.191361,0.583333,0.191361,-0.204715,0.333333,0.045768,2.220446e-16,-0.699485,-0.915966,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-06,0.431321,0.810036,-1.000000,0.810036,1.000000,1.000000,1.000000,-3.333333e-01,-1.000000,1.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
2020-07-07,-0.144338,0.869179,-1.000000,0.869179,1.000000,1.000000,0.491932,-4.444444e-01,-1.000000,1.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2020-07-08,0.595946,0.821685,0.558442,0.821685,-0.284555,1.000000,0.540333,-6.470588e-01,-0.466979,0.586364,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [212]:
temp_tesla['close'] = close['TSLA']
temp_tesla['open'] = open['TSLA']
temp_tesla['high'] = high['TSLA']
temp_tesla['low'] = low['TSLA']
temp_tesla['volume'] = np.log(volume['TSLA'])
temp_tesla['vwap'] = vwap['TSLA']
temp_tesla['returns'] = returns['TSLA']

In [213]:
temp_tesla.dropna(inplace=True)

In [214]:
temp_tesla

,A_0,A_1,A_2,A_3,A_4,A_5,A_6,A_7,A_8,A_9,...,y_A_7,y_A_8,y_A_9,close,open,high,low,volume,vwap,returns
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-22,-0.585260,0.531981,0.548387,0.531981,-0.339226,0.555556,-0.762320,-7.777778e-01,-1.000000,-0.882952,...,0.0,1.0,0.0,2.346000,2.304000,2.370000,2.284000,17.175990,2.333333,0.019409
2013-01-23,-0.678095,0.506168,-0.062500,0.506168,-0.343458,0.555556,-0.429030,-6.470588e-01,-1.000000,-0.964286,...,0.0,1.0,0.0,2.400000,2.334667,2.416000,2.330667,16.970999,2.382222,0.023018
2013-01-24,-0.678095,0.476078,-0.215190,0.476078,0.279617,0.333333,0.749645,-7.777778e-01,-1.000000,-0.693848,...,0.0,0.0,0.0,2.466000,2.400000,2.514667,2.389333,17.201797,2.456667,0.027500
2013-01-25,-0.205853,0.168249,0.555556,0.168249,0.576541,0.333333,-0.088115,-1.111111e-01,0.007737,-0.803175,...,0.0,0.0,1.0,2.465333,2.466667,2.502667,2.453333,16.776496,2.473778,-0.000271
2013-01-28,-0.206203,0.191361,0.583333,0.191361,-0.204715,0.333333,0.045768,2.220446e-16,-0.699485,-0.915966,...,0.0,0.0,0.0,2.535333,2.457333,2.580667,2.457333,17.209683,2.524444,0.028394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-02,0.431321,0.664981,-1.000000,0.664981,1.000000,1.000000,1.000000,1.000000e+00,-1.000000,1.000000,...,0.0,1.0,0.0,80.577332,81.431999,81.866669,79.040001,19.371379,80.494667,0.079517
2020-07-06,0.431321,0.810036,-1.000000,0.810036,1.000000,1.000000,1.000000,-3.333333e-01,-1.000000,1.000000,...,0.0,0.0,1.0,91.438667,85.112663,91.852669,84.402664,19.547390,89.231333,0.134794
2020-07-07,-0.144338,0.869179,-1.000000,0.869179,1.000000,1.000000,0.491932,-4.444444e-01,-1.000000,1.000000,...,0.0,0.0,0.0,92.657333,93.667336,95.300003,89.113998,19.591135,92.357112,0.013328


In [ ]:
temp_tesla.replace([-np.inf,np.inf],0 , inplace=True)

In [215]:
X = temp_tesla.drop(y_pred_col , axis=1)
y = temp_tesla[y_pred_col]

In [217]:
X

,A_0,A_1,A_2,A_3,A_4,A_5,A_6,A_7,A_8,A_9,close,open,high,low,volume,vwap,returns
Date,,,,,,,,,,,,,,,,,
2013-01-22,-0.585260,0.531981,0.548387,0.531981,-0.339226,0.555556,-0.762320,-7.777778e-01,-1.000000,-0.882952,2.346000,2.304000,2.370000,2.284000,17.175990,2.333333,0.019409
2013-01-23,-0.678095,0.506168,-0.062500,0.506168,-0.343458,0.555556,-0.429030,-6.470588e-01,-1.000000,-0.964286,2.400000,2.334667,2.416000,2.330667,16.970999,2.382222,0.023018
2013-01-24,-0.678095,0.476078,-0.215190,0.476078,0.279617,0.333333,0.749645,-7.777778e-01,-1.000000,-0.693848,2.466000,2.400000,2.514667,2.389333,17.201797,2.456667,0.027500
2013-01-25,-0.205853,0.168249,0.555556,0.168249,0.576541,0.333333,-0.088115,-1.111111e-01,0.007737,-0.803175,2.465333,2.466667,2.502667,2.453333,16.776496,2.473778,-0.000271
2013-01-28,-0.206203,0.191361,0.583333,0.191361,-0.204715,0.333333,0.045768,2.220446e-16,-0.699485,-0.915966,2.535333,2.457333,2.580667,2.457333,17.209683,2.524444,0.028394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-02,0.431321,0.664981,-1.000000,0.664981,1.000000,1.000000,1.000000,1.000000e+00,-1.000000,1.000000,80.577332,81.431999,81.866669,79.040001,19.371379,80.494667,0.079517
2020-07-06,0.431321,0.810036,-1.000000,0.810036,1.000000,1.000000,1.000000,-3.333333e-01,-1.000000,1.000000,91.438667,85.112663,91.852669,84.402664,19.547390,89.231333,0.134794
2020-07-07,-0.144338,0.869179,-1.000000,0.869179,1.000000,1.000000,0.491932,-4.444444e-01,-1.000000,1.000000,92.657333,93.667336,95.300003,89.113998,19.591135,92.357112,0.013328


In [2]:
import tensorflow as tf

def get_model(max_len):
    main_input = tf.keras.layers.Input(shape=(max_len,))
    
    m = (Dense(512, activation='tanh')) (main_input)
    m = (Dropout(0.2))(m)
    m =(Dense(256, activation='tanh'))(m)
    m = (Dropout(0.2))(m)
    m =(Dense(128, activation='tanh')) (m)
    m = (Dropout(0.2))(m)
    m =(Dense(64, activation='tanh')) (m)
    
    cnn = tf.keras.layers.Conv1D(128, 3, padding='same', strides=1, activation='relu')(main_input)
    cnn = tf.keras.layers.MaxPooling1D(pool_size=128)(cnn)
    flat = tf.keras.layers.Flatten()(cnn)
    flat = tf.keras.layers.Concatenate(axis=-1)([flat , m])
    drop = tf.keras.layers.Dropout(0.2)(flat)
    
    dense1 = tf.keras.layers.Dense(512, activation='relu')(drop)
    main_output = tf.keras.layers.Dense(1, activation='sigmoid')(dense1)
    model = tf.keras.Model(inputs=main_input, outputs=main_output)
    return model
model = get_model(X_train.shape[1])
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

NameError: name 'Dense' is not defined

In [233]:
x = Input(( 17 , ))

# m = (LSTM(17, input_shape=(17,0)))(x)
m = (Dense(512, activation='tanh')) (x)
m = (Dropout(0.2))(m)
m =(Dense(256, activation='tanh'))(m)
m = (Dropout(0.2))(m)
m =(Dense(128, activation='tanh')) (m)
m = (Dropout(0.2))(m)
m =(Dense(64, activation='tanh')) (m)
m =(Dense(128, activation='tanh'))(m)
m =(Dense(256, activation='tanh'))(m)
m = (Dense(512, activation='tanh'))(m)

t = tf.keras.layers.Concatenate(axis=-1)([x , m])
t =(Dense(256, activation='tanh'))(t)
f = (Dense(10, activation='softmax')) (t)

model = Model(inputs=x, outputs=f)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

val_df = pd.DataFrame()
all_df = pd.DataFrame()

all_y_df = pd.DataFrame()

for i in range(0 , len(X) , 30):
    tempx = X[i:i+30]
    tempy = y[i:i+30]
    X_train = tempx[:25]
    X_test = tempx[25:]
    y_train = tempy[:25]
    y_test = tempy[25:]

    model.fit(X_train , y_train, epochs=30 , verbose=1)
    try:
        temp = X_train.copy()
        temp = pd.concat([ temp , pd.DataFrame(model.predict(X_test) , columns=y_pred_col)], axis=1)
        val_df = pd.concat([val_df , temp] , axis=0)
        print("______________\n")
        print(model.evaluate(X_test , y_test))
        print("______________\n")
    except:
        ...
        print('error')
   # temp1 = tempx.copy()
  # temp1 = pd.concat([ all_df , pd.DataFrame(model.predict(tempx) , columns=y_pred_col)], axis=1)
  # all_df = pd.concat([all_df , temp1] , axis=0)

Epoch 1/30
1/1 [==============================] - 1s 1s/step - loss: 3.4366 - accuracy: 0.1200
Epoch 2/30
1/1 [==============================] - 0s 13ms/step - loss: 2.2103 - accuracy: 0.3200
Epoch 3/30
1/1 [==============================] - 0s 20ms/step - loss: 2.9902 - accuracy: 0.2000
Epoch 4/30
1/1 [==============================] - 0s 15ms/step - loss: 3.6016 - accuracy: 0.2400
Epoch 5/30
1/1 [==============================] - 0s 13ms/step - loss: 3.4245 - accuracy: 0.1600
Epoch 6/30
1/1 [==============================] - 0s 17ms/step - loss: 2.7876 - accuracy: 0.2000
Epoch 7/30
1/1 [==============================] - 0s 18ms/step - loss: 2.5882 - accuracy: 0.2000
Epoch 8/30
1/1 [==============================] - 0s 12ms/step - loss: 2.5573 - accuracy: 0.1600
Epoch 9/30
1/1 [==============================] - 0s 16ms/step - loss: 2.1942 - accuracy: 0.2000
Epoch 10/30
1/1 [==============================] - 0s 14ms/step - loss: 2.0437 - accuracy: 0.2400
Epoch 11/30
1/1 [==============

1/1 [==============================] - 0s 18ms/step - loss: 1.9407 - accuracy: 0.4000
Epoch 8/30
1/1 [==============================] - 0s 14ms/step - loss: 1.9394 - accuracy: 0.4000
Epoch 9/30
1/1 [==============================] - 0s 30ms/step - loss: 1.9383 - accuracy: 0.4000
Epoch 10/30
1/1 [==============================] - 0s 22ms/step - loss: 1.9375 - accuracy: 0.4000
Epoch 11/30
1/1 [==============================] - 0s 17ms/step - loss: 1.9368 - accuracy: 0.4000
Epoch 12/30
1/1 [==============================] - 0s 20ms/step - loss: 1.9361 - accuracy: 0.4000
Epoch 13/30
1/1 [==============================] - 0s 16ms/step - loss: 1.9352 - accuracy: 0.4000
Epoch 14/30
1/1 [==============================] - 0s 19ms/step - loss: 1.9334 - accuracy: 0.4000
Epoch 15/30
1/1 [==============================] - 0s 17ms/step - loss: 1.9352 - accuracy: 0.4000
Epoch 16/30
1/1 [==============================] - 0s 18ms/step - loss: 1.9344 - accuracy: 0.4000
Epoch 17/30
1/1 [=================

1/1 [==============================] - 0s 12ms/step - loss: 4.3669 - accuracy: 0.3200
Epoch 14/30
1/1 [==============================] - 0s 11ms/step - loss: 4.3378 - accuracy: 0.3200
Epoch 15/30
1/1 [==============================] - 0s 13ms/step - loss: 4.3097 - accuracy: 0.3200
Epoch 16/30
1/1 [==============================] - 0s 16ms/step - loss: 4.2842 - accuracy: 0.3200
Epoch 17/30
1/1 [==============================] - 0s 17ms/step - loss: 4.2589 - accuracy: 0.3200
Epoch 18/30
1/1 [==============================] - 0s 11ms/step - loss: 4.2371 - accuracy: 0.3200
Epoch 19/30
1/1 [==============================] - 0s 12ms/step - loss: 4.2135 - accuracy: 0.3200
Epoch 20/30
1/1 [==============================] - 0s 13ms/step - loss: 4.1969 - accuracy: 0.3200
Epoch 21/30
1/1 [==============================] - 0s 10ms/step - loss: 4.1729 - accuracy: 0.3200
Epoch 22/30
1/1 [==============================] - 0s 12ms/step - loss: 4.1660 - accuracy: 0.3200
Epoch 23/30
1/1 [===============

Epoch 1/30
1/1 [==============================] - 0s 20ms/step - loss: 2.6327 - accuracy: 0.3600
Epoch 2/30
1/1 [==============================] - 0s 18ms/step - loss: 2.3286 - accuracy: 0.3600
Epoch 3/30
1/1 [==============================] - 0s 17ms/step - loss: 2.2476 - accuracy: 0.3600
Epoch 4/30
1/1 [==============================] - 0s 23ms/step - loss: 2.2292 - accuracy: 0.3600
Epoch 5/30
1/1 [==============================] - 0s 20ms/step - loss: 2.2257 - accuracy: 0.3600
Epoch 6/30
1/1 [==============================] - 0s 19ms/step - loss: 2.2238 - accuracy: 0.3600
Epoch 7/30
1/1 [==============================] - 0s 23ms/step - loss: 2.2260 - accuracy: 0.3600
Epoch 8/30
1/1 [==============================] - 0s 17ms/step - loss: 2.2268 - accuracy: 0.3600
Epoch 9/30
1/1 [==============================] - 0s 19ms/step - loss: 2.2297 - accuracy: 0.3600
Epoch 10/30
1/1 [==============================] - 0s 16ms/step - loss: 2.2309 - accuracy: 0.3600
Epoch 11/30
1/1 [============

1/1 [==============================] - 0s 22ms/step - loss: 3.9443 - accuracy: 0.1200
Epoch 8/30
1/1 [==============================] - 0s 26ms/step - loss: 3.9007 - accuracy: 0.1200
Epoch 9/30
1/1 [==============================] - 0s 24ms/step - loss: 3.8651 - accuracy: 0.1200
Epoch 10/30
1/1 [==============================] - 0s 20ms/step - loss: 3.8362 - accuracy: 0.1200
Epoch 11/30
1/1 [==============================] - 0s 21ms/step - loss: 3.8128 - accuracy: 0.1200
Epoch 12/30
1/1 [==============================] - 0s 24ms/step - loss: 3.7940 - accuracy: 0.1200
Epoch 13/30
1/1 [==============================] - 0s 26ms/step - loss: 3.7788 - accuracy: 0.1200
Epoch 14/30
1/1 [==============================] - 0s 19ms/step - loss: 3.7665 - accuracy: 0.1200
Epoch 15/30
1/1 [==============================] - 0s 16ms/step - loss: 3.7564 - accuracy: 0.1200
Epoch 16/30
1/1 [==============================] - 0s 20ms/step - loss: 3.7483 - accuracy: 0.1200
Epoch 17/30
1/1 [=================

...

1/1 [==============================] - 0s 13ms/step - loss: 2.7666 - accuracy: 0.2400
Epoch 8/30
1/1 [==============================] - 0s 16ms/step - loss: 2.7695 - accuracy: 0.2400
Epoch 9/30
1/1 [==============================] - 0s 12ms/step - loss: 2.7712 - accuracy: 0.2400
Epoch 10/30
1/1 [==============================] - 0s 13ms/step - loss: 2.7739 - accuracy: 0.2400
Epoch 11/30
1/1 [==============================] - 0s 14ms/step - loss: 2.7754 - accuracy: 0.2400
Epoch 12/30
1/1 [==============================] - 0s 14ms/step - loss: 2.7779 - accuracy: 0.2400
Epoch 13/30
1/1 [==============================] - 0s 12ms/step - loss: 2.7791 - accuracy: 0.2400
Epoch 14/30
1/1 [==============================] - 0s 15ms/step - loss: 2.7817 - accuracy: 0.2400
Epoch 15/30
1/1 [==============================] - 0s 12ms/step - loss: 2.7825 - accuracy: 0.2400
Epoch 16/30
1/1 [==============================] - 0s 13ms/step - loss: 2.7854 - accuracy: 0.2400
Epoch 17/30
1/1 [=================

1/1 [==============================] - 0s 11ms/step - loss: 4.6200 - accuracy: 0.1600
Epoch 14/30
1/1 [==============================] - 0s 10ms/step - loss: 4.6059 - accuracy: 0.1600
Epoch 15/30
1/1 [==============================] - 0s 12ms/step - loss: 4.5925 - accuracy: 0.1600
Epoch 16/30
1/1 [==============================] - 0s 14ms/step - loss: 4.5797 - accuracy: 0.1600
Epoch 17/30
1/1 [==============================] - 0s 14ms/step - loss: 4.5676 - accuracy: 0.1600
Epoch 18/30
1/1 [==============================] - 0s 14ms/step - loss: 4.5560 - accuracy: 0.1600
Epoch 19/30
1/1 [==============================] - 0s 15ms/step - loss: 4.5449 - accuracy: 0.1600
Epoch 20/30
1/1 [==============================] - 0s 11ms/step - loss: 4.5344 - accuracy: 0.1600
Epoch 21/30
1/1 [==============================] - 0s 14ms/step - loss: 4.5243 - accuracy: 0.1600
Epoch 22/30
1/1 [==============================] - 0s 11ms/step - loss: 4.5147 - accuracy: 0.1600
Epoch 23/30
1/1 [===============

Epoch 1/30
1/1 [==============================] - 0s 19ms/step - loss: 3.3025 - accuracy: 0.3200
Epoch 2/30
1/1 [==============================] - 0s 14ms/step - loss: 3.1413 - accuracy: 0.2000
Epoch 3/30
1/1 [==============================] - 0s 13ms/step - loss: 3.0835 - accuracy: 0.2000
Epoch 4/30
1/1 [==============================] - 0s 28ms/step - loss: 3.0679 - accuracy: 0.2000
Epoch 5/30
1/1 [==============================] - 0s 23ms/step - loss: 3.0576 - accuracy: 0.2000
Epoch 6/30
1/1 [==============================] - 0s 19ms/step - loss: 3.0523 - accuracy: 0.2000
Epoch 7/30
1/1 [==============================] - 0s 27ms/step - loss: 3.0486 - accuracy: 0.2000
Epoch 8/30
1/1 [==============================] - 0s 21ms/step - loss: 3.0462 - accuracy: 0.2000
Epoch 9/30
1/1 [==============================] - 0s 16ms/step - loss: 3.0444 - accuracy: 0.2000
Epoch 10/30
1/1 [==============================] - 0s 13ms/step - loss: 3.0431 - accuracy: 0.2000
Epoch 11/30
1/1 [============

1/1 [==============================] - 0s 12ms/step - loss: 3.2582 - accuracy: 0.3200
Epoch 8/30
1/1 [==============================] - 0s 13ms/step - loss: 3.1998 - accuracy: 0.3200
Epoch 9/30
1/1 [==============================] - 0s 17ms/step - loss: 3.1416 - accuracy: 0.3200
Epoch 10/30
1/1 [==============================] - 0s 15ms/step - loss: 3.0855 - accuracy: 0.3200
Epoch 11/30
1/1 [==============================] - 0s 17ms/step - loss: 3.0290 - accuracy: 0.3200
Epoch 12/30
1/1 [==============================] - 0s 11ms/step - loss: 2.9745 - accuracy: 0.3200
Epoch 13/30
1/1 [==============================] - 0s 10ms/step - loss: 2.9192 - accuracy: 0.3200
Epoch 14/30
1/1 [==============================] - 0s 9ms/step - loss: 2.8652 - accuracy: 0.3200
Epoch 15/30
1/1 [==============================] - 0s 12ms/step - loss: 2.8101 - accuracy: 0.3200
Epoch 16/30
1/1 [==============================] - 0s 12ms/step - loss: 2.7559 - accuracy: 0.3200
Epoch 17/30
1/1 [==================

1/1 [==============================] - 0s 11ms/step - loss: 5.4749 - accuracy: 0.3500
Epoch 14/30
1/1 [==============================] - 0s 15ms/step - loss: 5.4708 - accuracy: 0.3500
Epoch 15/30
1/1 [==============================] - 0s 14ms/step - loss: 5.4672 - accuracy: 0.3500
Epoch 16/30
1/1 [==============================] - 0s 10ms/step - loss: 5.4642 - accuracy: 0.3500
Epoch 17/30
1/1 [==============================] - 0s 17ms/step - loss: 5.4615 - accuracy: 0.3500
Epoch 18/30
1/1 [==============================] - 0s 14ms/step - loss: 5.4592 - accuracy: 0.3500
Epoch 19/30
1/1 [==============================] - 0s 13ms/step - loss: 5.4571 - accuracy: 0.3500
Epoch 20/30
1/1 [==============================] - 0s 13ms/step - loss: 5.4553 - accuracy: 0.3500
Epoch 21/30
1/1 [==============================] - 0s 12ms/step - loss: 5.4538 - accuracy: 0.3500
Epoch 22/30
1/1 [==============================] - 0s 11ms/step - loss: 5.4524 - accuracy: 0.3500
Epoch 23/30
1/1 [===============

In [236]:
b = pd.DataFrame(model.predict(X))

59/59 [==============================] - 0s 3ms/step


In [258]:
a = list((temp_tesla['close'] - temp_tesla['open'])/temp_tesla['open'])
b = list(result_df.sum(axis=1))
s = 0
for i in range(len(a)):
    s += a[i]*b[i]
print(s)

-1.1520137792605785


In [257]:
result_data = [[float((tesla.iloc[i, j])*b.iloc[i, j]) for j in range(len(b.columns))] for i in range(len(b))]
result_df = pd.DataFrame(result_data , columns=pred_tesla.columns)
result_df.set_index(b.index , inplace=True)
result_df.sum(axis=1)

0      -0.769172
1      -0.790956
2      -0.732957
3       0.015226
4      -0.548143
          ...   
1875    0.050007
1876    0.059938
1877    0.063415
1878    0.379767
1879   -0.300875
Length: 1880, dtype: float64

In [ ]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 17)]         0           []                               
                                                                                                  
 dense_61 (Dense)               (None, 256)          4608        ['input_11[0][0]']               
                                                                                                  
 dense_62 (Dense)               (None, 128)          32896       ['dense_61[0][0]']               
                                                                                                  
 dense_63 (Dense)               (None, 64)           8256        ['dense_62[0][0]']               
                                                                                            